In [ ]:
import h5py

import pandas as pd
import numpy as np
from collections import defaultdict
import requests
from Bio import SeqIO, SearchIO
import umap

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import matplotlib.pyplot as plt

## Proteome visualization

In [5]:
# Declaring file paths and parameters.

per_protein_embeddings_path = './Data/protein_embeddings_uniprot.h5'

proteome_dataframe_path = './Data/proteome.parquet'
kinase_domains_dataframe_path = './Data/distinct_kinase_domain_embeddings.parquet'

n_neighbors_values = [5, 15, 30, 45]
min_dist_values = [0.01, 0.1, 0.5, 1]

In [3]:
# Utility functions.

def run_UMAP_grid_scan(embeddings, n_neighbors_params=n_neighbors_values, min_dist_params=min_dist_values):
    """
    Perform grid scan for UMAP hyperparameters.
    
    Args:
        embeddings (list): List of per protein embedding vectors.
        n_neighbors_params (list): List of inputs for the n_neighbors argument of UMAP.
        min_dist_params (list): List of inputs for the min_dist argument of UMAP.
        
    Returns:
        umap_results (dict): A dictionary with (n_neighbors, min_dist) tuples as keys and
                             2D UMAP embeddings as values.
    """
    
    umap_results = {}
    for n_neighbors in n_neighbors_values:
        for min_dist in min_dist_values:
            umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)  # fixed seed for reproducibility
            embedding_2d = umap_model.fit_transform(embeddings)

            umap_results[(n_neighbors, min_dist)] = embedding_2d

    return umap_results


def add_colors_to_proteome_df(terms_or_keywords, proteome_df, term_type='Keywords'):
    """
    Adds a color column to the proteome DataFrame based on matching GO terms or keywords.
    
    Args:
        terms_or_keywords (list): List of GO terms or keywords to match.
        proteome_df (pd.DataFrame): DataFrame containing protein information.
        term_type (str): Column to search for terms/keywords ('GO_terms' or 'Keywords').
        
    Returns:
        proteome_df (pd.DataFrame): The updated DataFrame with a 'Color' column.
    """

    unique_terms = set(terms_or_keywords)
    colors = px.colors.qualitative.Plotly
    color_palette = {term: colors[i % len(colors)] for i, term in enumerate(unique_terms)}

    default_color = 'lightgrey'

    proteome_df['Color'] = default_color

    for term in terms_or_keywords:
        mask = proteome_df[term_type].astype(str).str.contains(term, case=False, na=False)
        proteome_df.loc[mask, 'Color'] = color_palette[term]

    return proteome_df


def show_UMAP_grid_scan(umap_results, hover_names, color, marker_size, *params):
    """
    Visualizes the grid scan of different UMAP model parameters using subplots.

    Args:
        umap_results (dict): A dictionary with (n_neighbors, min_dist) tuples as keys and
                             2D UMAP embeddings as values.
        hover_names (list): A list of names to display when hovering over points in the scatter plots.
        color (str or None): A color specification to apply to the points (e.g., a list of colors or a column name).
        marker_size (int): The size of the scatter plot markers.
        *params (lists): Two lists specifying the UMAP parameters:
                         - `params[0]`: List of `n_neighbors` values.
                         - `params[1]`: List of `min_dist` values.

    Returns:
        None: Displays the UMAP projections as a grid of subplots.
    """
        
    fig = sp.make_subplots(
        rows=len(params[0]), 
        cols=len(params[1]),
        subplot_titles=[f"n_neighbors={n}, min_dist={d}" for n in params[0] for d in params[1]],
        horizontal_spacing=0.1, vertical_spacing=0.1
        )
    for i, n_neighbors in enumerate(params[0]):
        for j, min_dist in enumerate(params[1]):
            embedding_2d = umap_results[(n_neighbors, min_dist)]
            
            if color:
                scatter = px.scatter(
                    x=embedding_2d[:, 0], 
                    y=embedding_2d[:, 1],
                    hover_name=hover_names,
                    color=color,
                    labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
                )
            else:
                    scatter = px.scatter(
                    x=embedding_2d[:, 0], 
                    y=embedding_2d[:, 1],
                    hover_name=hover_names,
                    labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
                )

            scatter.update_traces(marker=dict(size=marker_size))

            for trace in scatter['data']:
                trace.showlegend = False
                fig.add_trace(trace, row=i+1, col=j+1)

    fig.update_layout(
        height=1600, 
        width=1600,
        title_text="UMAP Projections with Different Hyperparameters",
        dragmode='zoom'
    )

    fig.show()

In [ ]:
protein_ids = []
embeddings = []

with h5py.File(per_protein_embeddings_path, "r") as file:
    print(f"Number of entries: {len(file.items())}")
    for sequence_id, embedding in file.items():
        if sequence_id:
            protein_ids.append(sequence_id)
            embeddings.append(np.array(embedding))

    assert len(file.items()) == len(embeddings)

per_protein_embeddings = np.array(embeddings)
id_to_embedding = dict(zip(protein_ids, per_protein_embeddings))

# Map embeddings to DataFrame entries by 'Uniprot ID'
proteome_df = pd.read_parquet(proteome_dataframe_path)
proteome_df['embedding'] = proteome_df['Uniprot ID'].map(id_to_embedding)

In [ ]:
umap_results = run_UMAP_grid_scan(proteome_df['embedding'].tolist())

In [ ]:
proteome_df = add_colors_to_proteome_df(['Kinase', 'G-protein coupled receptor'], proteome_df, term_type='Keywords')
show_UMAP_grid_scan(umap_results, proteome_df.GeneName.tolist(), proteome_df['Color'].tolist(), 2, n_neighbors_values, min_dist_values)

## Kinome visualization

#### Preprocessing of embeddings

In [22]:
# Declaring file paths and parameters.

proteome_fasta_path = './Data/UP000005640_9606.fasta'
hmmerscan_results_path = './Data/proteome_hmmer_results.out'
per_residue_embeddings_path = './Data/per-residue.h5'  # TODO download file form https://www.uniprot.org/help/downloads

# List of kinase specific GO terms present in the mapping supplied by https://current.geneontology.org/ontology/external2go/pfam2go
kinase_go_terms = [
        'GO:0016301', 'GO:0004672', 'GO:0052742', 
        'GO:0004618', 'GO:0004743', 'GO:0004797', 
        'GO:0003872', 'GO:0004674', 'GO:0004797', 
        'GO:0004611', 'GO:0003848', 'GO:0003873',
        'GO:0004417', 'GO:0043752', 'GO:0004550',
        'GO:0004340', 'GO:0004673', 'GO:0004371',
        'GO:0004594', 'GO:0004788', 'GO:0004631',
        'GO:0008671', 'GO:0035299', 'GO:0008531',
        'GO:0008772', 'GO:0004797', 'GO:0009029',
        'GO:0004611', 'GO:0008887'
        ]

# List of kinase domains that do not serve are not directly responsible for the kinases catalytic activity.
domains_to_exclude = [
        'PF06414', 'PF12063', 'PF11629',
        'PF08826', 'PF05191', 'PF02807',
        'PF12202', 'PF00433', 'PF09036',
        'PF12605', 'PF09202', 'PF11640',
        'PF08926', 'PF02816', 'PF08064',
        'PF02734', 'PF15785'
        ]

In [16]:
# Utility functions.

def parse_pfam_go_file(url="https://current.geneontology.org/ontology/external2go/pfam2go"):
    """
    Parses the PFAM to GO-term mapping file from a specified URL.

    Args:
        url (str): The URL of the PFAM-to-GO mapping file.
                   Default is "https://current.geneontology.org/ontology/external2go/pfam2go".

    Returns:
        defaultdict: A dictionary mapping GO terms to sets of PFAM accessions.
    """
    
    go_to_pfam = defaultdict(set)

    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    lines = response.text.splitlines()

    for line in lines:
        if line.startswith("Pfam:"):
            pfam_part, go_part = line.strip().split(' > GO:')
            pfam_accession = pfam_part.split()[0].replace("Pfam:", "")
            go_term = go_part.split(' ; ')[1]
            go_to_pfam[go_term].add(pfam_accession)

    return go_to_pfam


def parse_hmmer_results(hmmer_results_file):
    """
    Parses an HMMER domtab result file.

    Args:
        hmmer_results_file (str): Path to the HMMER domtab result file.

    Returns:
        iterator: An iterator of parsed HMMER search results.
    """

    parsed_hmmer_file = SearchIO.parse(hmmer_results_file, "hmmscan3-domtab")
    return parsed_hmmer_file


def filter_domains_by_GO(parsed_hmmer_file, go_to_pfam, target_go_terms, exluded_domains):
    """
    Filters the parsed HMMER file to extract domains associated with specific GO terms.

    Args:
        parsed_hmmer_file (iterator): Parsed HMMER search results.
        go_to_pfam (dict): Dictionary mapping GO terms to PFAM accessions.
        target_go_terms (list): List of GO terms to filter by.
        exluded_domains (list): List of PFAM accessions to exclude from the results.

    Returns:
        list: A list of tuples containing filtered domain information:
              (query_id, pfam_accession, env_start, env_end, query_start, query_end, target_description).
    """

    filtered_domains = []
    for queryresult in parsed_hmmer_file:
        for hit in queryresult.hits:
            for hsp in hit.hsps:
                pfam_accession = hit.accession.split('.')[0]
                if hsp.evalue <= 1e-5:  # Threshold for domains with significant matches to a PFAM family.
                    for go_term in target_go_terms:
                        if (pfam_accession in go_to_pfam[go_term]) and (pfam_accession not in exluded_domains):
                            target_description = hit.description
                            filtered_domains.append((queryresult.id, pfam_accession, hsp.env_start, hsp.env_end, hsp.query_start, hsp.query_end, target_description))
    return filtered_domains


def parse_fasta_file(fasta_file):
    """
    Parses a multi-FASTA file and creates a mapping of UniProt IDs to sequences.

    Args:
        fasta_file (str): Path to the FASTA file.

    Returns:
        dict: A dictionary mapping UniProt IDs to sequences.
    """

    sequences = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences[record.id] = record.seq
    return sequences


def extract_domain_sequences(filtered_domains, sequences):
    """
    Extracts domain sequences from full-length protein sequences based on domain boundaries.

    Args:
        filtered_domains (list): A list of tuples containing filtered domain information:
                                 (query_id, pfam_accession, env_start, env_end, query_start, query_end, target_description).
        sequences (dict): A dictionary mapping UniProt IDs to full-length sequences.

    Returns:
        list: A list of tuples containing extracted domain sequences:
              (query_id, protein_id, entry_name, pfam_accession, env_start, env_end,
              query_start, query_end, domain_sequence, target_description).
    """
    
    extracted_sequences = []
    for domain in filtered_domains:
        query_id, pfam_accession, env_start, env_end, query_start, query_end, target_description = domain
        sequence = sequences.get(query_id)
        if sequence:
            domain_seq = sequence[env_start:env_end]
            extracted_sequences.append((query_id,query_id.split('|')[1],query_id.split('|')[2], pfam_accession, env_start, env_end, query_start, query_end, str(domain_seq), target_description))
    return extracted_sequences

In [ ]:
go_to_pfam_dict = parse_pfam_go_file()
parsed_hmmer_results = parse_hmmer_results(hmmerscan_results_path)
filtered_domains = filter_domains_by_GO(parsed_hmmer_results, go_to_pfam_dict, kinase_go_terms, domains_to_exclude)
human_proteome_sequences = parse_fasta_file(proteome_fasta_path)
extracted_sequences = extract_domain_sequences(filtered_domains, human_proteome_sequences)

kinase_domain_df = pd.DataFrame(extracted_sequences, 
                                columns=
                                    [
                                    'Query ID','Uniprot ID','Protein', 
                                    'Pfam Accession', 'Env Start', 'Env End', 
                                    'Query Start', 'Query End', 'Domain Sequence', 
                                    'Domain Description'
                                    ]
                                )
kinase_domain_df.head()

In [18]:
# To account for cases where a single domain is matched by multiple HMM models (resulting in multiple PFAM IDs), 
# this logic selectively retains non-overlapping domains.

def has_overlap(domain1, domain2):
    """
    Checks if two domains overlap based on their start and end positions.

    Args:
        domain1 (tuple): The start and end positions of the first domain (start1, end1).
        domain2 (tuple): The start and end positions of the second domain (start2, end2).

    Returns:
        bool: True if the domains overlap, False otherwise.
    """

    start1, end1 = domain1
    start2, end2 = domain2
    return max(start1, start2) <= min(end1, end2)


def filter_nonoverlapping_domains(group):
    """
    Filters out overlapping domains for each UniProt ID group.

    Args:
        group (DataFrame): A DataFrame containing domain information with columns 'Env Start' and 'Env End'.

    Returns:
        DataFrame: A DataFrame containing non-overlapping domains.
    """

    selected_rows = []
    for idx, row in group.iterrows():
        current_domain = (row['Env Start'], row['Env End'])
        overlap = any(has_overlap(current_domain, (group.loc[i, 'Env Start'], group.loc[i, 'Env End'])) for i in selected_rows)
        
        if not overlap:
            selected_rows.append(idx)

    return group.loc[selected_rows]


def crop_embedding(row):
    """
    Crops the embedding of a protein domain based on specified domain boundaries.

    Args:
        row (Series): A pandas Series containing:
                      - 'Uniprot ID' (str): The UniProt identifier.
                      - 'Query Start' (int): Start position of the domain in the query sequence.
                      - 'Query End' (int): End position of the domain in the query sequence.
                      - 'Embedding' (ndarray): The embedding matrix (shape: k x 1024) for the sequence.

    Returns:
        ndarray or None: The cropped embedding matrix based on domain boundaries.
                         Returns None if the embedding is missing.

    Raises:
        AssertionError: If the cropped embedding length doesn't match the expected domain length.
    """
    
    uniprot_id = row['Uniprot ID']
    query_start = row['Query Start']
    query_end = row['Query End']
    embedding = row['Embedding']

    if embedding is not None:
        # Crop the embedding based on the domain boundaries
        cropped_embedding = embedding[query_start:query_end+1, :]  # Crop rows, keep all columns (1024)

        domain_length = query_end - query_start + 1
        assert cropped_embedding.shape[0] == domain_length, (
            f"Length mismatch for {uniprot_id}: Domain length is {domain_length} "
            f"but cropped embedding length is {cropped_embedding.shape[0]}"
        )
        return cropped_embedding
    else:
        return None

In [ ]:
distinct_domains_df = kinase_domain_df.groupby('Uniprot ID', group_keys=False).apply(filter_nonoverlapping_domains, include_groups=True)
distinct_domains_df.head()

In [ ]:
distinct_domains_df['Domain Length'] = distinct_domains_df['Env End'] - distinct_domains_df['Env Start']

fig = px.histogram(distinct_domains_df, x='Domain Length', nbins=100, title="Distribution of Kinase Domain Lengths")
fig.update_layout(
    xaxis_title="Domain Length",
    yaxis_title="Frequency",
    template="plotly_white",
    width=600,
    height=400
)

fig.show()

In [ ]:
embedding_map = {}

with h5py.File(per_residue_embeddings_path, "r") as file:
    print(f"Number of entries: {len(file.items())}")
    for sequence_id, embedding in file.items():
        embedding_map[sequence_id] = np.array(embedding)

# Add embeddings to respective domains and crop them to fit the domain length.
distinct_domains_df['Embedding'] = distinct_domains_df['Uniprot ID'].map(embedding_map)
distinct_domains_df.dropna(subset=['Embedding'], inplace=True)
distinct_domains_df['Cropped Embedding'] = distinct_domains_df.apply(crop_embedding, axis=1)

In [24]:
# In order to allow for the application of dimensionalty reduction algorithms, the domain embeddings have to be padded.
# Each row of the embedding matrix represents a single residue with a vector of size 1024.

def pad_with_zeros(row, max_length):
    """Pad domain embeddings with zero vectors of shape (1, 1024)."""

    cropped_embedding = row['Cropped Embedding']

    if cropped_embedding is not None:
        padding_length = max_length - cropped_embedding.shape[0]
        
        if padding_length > 0:
            padding = np.zeros((padding_length, 1024), dtype=np.float32)
            padded_embedding = np.vstack([cropped_embedding.astype(np.float32), padding])
        else:
            padded_embedding = cropped_embedding.astype(np.float32)
        
        return padded_embedding.flatten()
    else:
        return None
    

def pad_with_mean(row, max_length):
    """Pad domain embeddings with column-wise mean vectors of shape (1, 1024)."""

    cropped_embedding = row['Cropped Embedding']

    if cropped_embedding is not None:
        padding_length = max_length - cropped_embedding.shape[0]

        if padding_length > 0:
            mean_vector = np.mean(cropped_embedding, axis=0).astype(np.float32)
            padding = np.tile(mean_vector, (padding_length, 1))
            padded_embedding = np.vstack([cropped_embedding.astype(np.float32), padding])
        else:
            padded_embedding = cropped_embedding.astype(np.float32)
        
        return padded_embedding.flatten()
    else:
        return None


def create_identifier(row):
    protein = row['Protein'].replace('_HUMAN', '')
    return f"{protein}_{row['Pfam Accession']}_{row['Env Start']}-{row['Env End']}"
    

max_length = max([embedding.shape[0] for embedding in distinct_domains_df['Cropped Embedding'] if embedding is not None])

distinct_domains_df['Linearized Zeros Embedding'] = distinct_domains_df.apply(lambda row: pad_with_zeros(row, max_length), axis=1)
distinct_domains_df['Linearized Mean Embedding'] = distinct_domains_df.apply(lambda row: pad_with_mean(row, max_length), axis=1)
distinct_domains_df['Identifier'] = distinct_domains_df.apply(create_identifier, axis=1)

In [25]:
# Export dataframe, if the autoencoder is run on another machine.
distinct_domains_df.drop(axis=1, columns=['Embedding', 'Cropped Embedding'], inplace=True)
distinct_domains_df.to_parquet('./Data/distinct_kinase_domain_embeddings.parquet')

The autoencoders can be trained on the linearized kinase domain embeddings now. Please refer to the 'model_training.ipynb' notebook.

In [ ]:
# Load the learned embeddings.
distinct_domains_df = pd.read_parquet('./Data/distinct_kinase_domain_embeddings_with_latents.parquet')
merged_df = pd.merge(proteome_df, distinct_domains_df, on='Uniprot ID', how='inner')

print(f"Number of entries in the merged DataFrame: {len(merged_df)}")
merged_df.head()

In [27]:
# Classify and color the domains.
STK_terms = ['GO:0004674']
TK_terms = ['GO:0004713', 'GO:0004714', 'GO:0007169']

# Function to classify domain based on GO terms
def classify_domain(go_terms):
    if any(term in STK_terms for term in go_terms):
        return 'STK'
    elif any(term in TK_terms for term in go_terms):
        return 'TK'
    else:
        return 'Other'

# Apply classification to the DataFrame
# Ensure GO_terms are all strings before splitting
merged_df['GO_terms'] = merged_df['GO_terms'].apply(lambda x: ','.join(x) if isinstance(x, np.ndarray) else x)
merged_df['GO_terms'] = merged_df['GO_terms'].astype(str)  # Convert all entries to strings

# Now apply the split and classification
merged_df['GO_terms'] = merged_df['GO_terms'].apply(lambda x: x.split(','))
merged_df['Domain_Type'] = merged_df['GO_terms'].apply(classify_domain)

In [28]:
def run_UMAP_on_multiple_embeddings(embedding_arrays, subplot_titles, n_neighbors=30, min_dist=0.75):
    """
    Run UMAP on different embedding types and store the results with subplot titles as keys.

    Parameters:
    - embedding_arrays (list of np.ndarray): List of embedding arrays.
    - subplot_titles (list of str): Titles for each embedding type.
    - n_neighbors (int): UMAP n_neighbors parameter.
    - min_dist (float): UMAP min_dist parameter.

    Returns:
    - umap_results (dict): Dictionary with subplot titles as keys and 2D UMAP embeddings as values.
    """

    umap_results = {}
    for embeddings, title in zip(embedding_arrays, subplot_titles):
        umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)  # fixed seed for reproducibility
        embedding_2d = umap_model.fit_transform(embeddings)
        
        # Store the result with the subplot title as the key
        umap_results[title] = embedding_2d

    return umap_results

In [ ]:
zeros_embeddings_array = np.array(merged_df['Linearized Zeros Embedding'].tolist())
mean_embeddings_array = np.array(merged_df['Linearized Mean Embedding'].tolist())
small_latent_array = np.array(merged_df['Latent Small Embedding'].tolist())
large_latent_array = np.array(merged_df['Latent Large Embedding'].tolist())

embedding_arrays = [
    zeros_embeddings_array, 
    mean_embeddings_array, 
    small_latent_array, 
    large_latent_array
]

subplot_titles = [
    "UMAP on Zeros Padding", 
    "UMAP on Mean Padding", 
    "UMAP on Small Latent (512)", 
    "UMAP on Large Latent (768)"
]

umap_results = run_UMAP_on_multiple_embeddings(embedding_arrays, subplot_titles)

In [ ]:
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=subplot_titles,
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

for idx, key in enumerate(subplot_titles):

    scatter = px.scatter(
        x=umap_results[key][:, 0], 
        y=umap_results[key][:, 1],
        color=merged_df['Domain_Type'],
        hover_name=distinct_domains_df['Identifier'],
        labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
    )
    scatter.update_traces(marker=dict(size=4))
    
    # Determine the row and column for subplot placement
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    for trace in scatter['data']:
        trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)

fig.update_layout(
    height=800, 
    width=800,
    title_text="UMAP Projections: Zeros, Mean, Small Latent (512), and Large Latent (768)",
    dragmode='zoom',
    showlegend=True,
    legend=dict(
        x=1.05,
        y=0.5,
        traceorder='normal',
        font=dict(size=12),
        borderwidth=2
    )
)

fig.show()

In [ ]:
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=subplot_titles,
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

sorted_identifiers = sorted(merged_df['Identifier'].unique())

# Generate a color gradient across the sorted identifiers to color proteins with alphabetically similar names with similar colors.
cmap = plt.get_cmap("viridis")
color_gradient = [cmap(i / (len(sorted_identifiers) - 1)) for i in range(len(sorted_identifiers))]
color_dict = {identifier: f'rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, {a})' for identifier, (r, g, b, a) in zip(sorted_identifiers, color_gradient)}

for idx, key in enumerate(subplot_titles):

    scatter = px.scatter(
        x=umap_results[key][:, 0], 
        y=umap_results[key][:, 1],
        color=merged_df['Identifier'],
        color_discrete_map=color_dict,
        hover_name=merged_df['Identifier'],
        labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
    )
    scatter.update_traces(marker=dict(size=4))
    
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    for trace in scatter['data']:
        trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)

fig.update_layout(
    height=800, 
    width=800,
    title_text="UMAP Projections: Zeros, Mean, Small Latent (512), and Large Latent (768)",
    dragmode='zoom'
)

fig.show()